---
toc: true
comments: true
layout: notebook
title: User Profile JavaScript/HTML Lesson
description: The College Board testing language is Pseudo Code.  Pseudo mean kind-of language that we will compare to Python.
courses: { csp: {week: 19} }
type: hacks
---

# User Profile JavaScript/HTML Lesson

## 